# Lateral Movement Threat Hunting - Anomalous RDP Logon

A remote desktop logon, through RDP, may be typical of a system administrator or IT support, but only from select workstations. Monitoring remote desktop logons and comparing to known/approved originating systems can detect lateral movement of an adversary. 

## Use Case
### Objective
Identify anomalous RDP logon event. An event may be considered anomalous if it is for example occured outside of normal  time(e.g. office hour) or if it is being done from suspicious Ip address.

### Logs Requirements
Microsoft Windows Sysmon

### References
MITRE references: https://car.mitre.org/analytics/CAR-2016-04-005/ <br>
evtx: https://github.com/sbousseaden/EVTX-ATTACK-SAMPLES <br>
geoip2 database: https://dev.maxmind.com/geoip/geoip2/downloadable/

In [1]:
import evtx
import xml.etree.ElementTree as ET
import json

In [2]:
evtx_file = "logs/Lateral Movement/DE_RDP_Tunneling_4624.evtx"

In [3]:
parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records_json())

Success_RDP = []

for pj in parse_json:
    if "4624" in pj['data'] and "\"LogonType\": 10" in pj['data']:
        Success_RDP+=[pj]
        print(pj['data'])

{
  "Event": {
    "#attributes": {
      "xmlns": "http://schemas.microsoft.com/win/2004/08/events/event"
    },
    "EventData": {
      "AuthenticationPackageName": "Negotiate",
      "IpAddress": "127.0.0.1",
      "IpPort": "49164",
      "KeyLength": 0,
      "LmPackageName": "-",
      "LogonGuid": "00000000-0000-0000-0000-000000000000",
      "LogonProcessName": "User32 ",
      "LogonType": 10,
      "ProcessId": "0x658",
      "ProcessName": "C:\\Windows\\System32\\winlogon.exe",
      "SubjectDomainName": "EXAMPLE",
      "SubjectLogonId": "0x3e7",
      "SubjectUserName": "PC02$",
      "SubjectUserSid": "S-1-5-18",
      "TargetDomainName": "PC02",
      "TargetLogonId": "0x45120",
      "TargetUserName": "IEUser",
      "TargetUserSid": "S-1-5-21-3583694148-1414552638-2922671848-1000",
      "TransmittedServices": "-",
      "WorkstationName": "PC02"
    },
    "System": {
      "Channel": "Security",
      "Computer": "PC02.example.corp",
      "Correlation": null,
     

<h2>RDP outside office hour</h2>

In [4]:
from datetime import datetime

Defining office hour filter

In [5]:
sh = 8  #office start hour
sm = 0  #office start minute
eh = 15 #office end hour
em = 0  #office end minute

In [9]:
for SRDP in Success_RDP:
    SRDP_time = datetime.strptime(SRDP['timestamp'], "%Y-%m-%d %H:%M:%S.%f %Z")
    if ((SRDP_time < SRDP_time.replace(hour=sh, minute=sm)) or (SRDP_time > SRDP_time.replace(hour=eh, minute=em))):
        print("Detected RDP logon at", SRDP['timestamp'])
#         print(SRDP['data'])

Detected RDP logon at 2019-02-13 15:27:53.653483 UTC


<h2>RDP Login Source Outside of Indonesia</h2>
<br>
As the infrastructure of the company reside in Indonesia, the access should be made from Indonesia as well. The geoip2 database from MaxMind could be utilizied to find out the accessing IP's geo location.

In [7]:
#import geoip library and read the geoip2 database
import geoip2.database
reader = geoip2.database.Reader('logs/Lateral Movement/GeoLocationDB/GeoLite2-Country.mmdb')

In [8]:
for SRDP in Success_RDP:
    try: #try clause in case no Ip address in the record
        SRDP_Ip = json.loads(SRDP['data'])['Event']['EventData']['IpAddress']
        try: #try clause in case there's no matchin ip address in the database
            response = reader.country(SRDP_Ip)

            if response.country.iso_code != 'ID':
                print("IP =", SRDP_Ip, response.country.name)
        except geoip2.errors.AddressNotFoundError:
            print("The address 127.0.0.1 is not in the database.")
    except:
        print("No IP address detected")

The address 127.0.0.1 is not in the database.


<b>notes</b>
<br>
The geoip2 library could also utilize MaxMind geoip2 web services. See http://dev.maxmind.com/geoip/geoip2/web-services for the GeoIP2 Precision web service docs.